# Neighboring Venues Around Cities in Hudson County of NJ

In [1]:
import pandas as pd
import numpy as np
import requests

url = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=NJ&rows=762&facet=state&facet=timezone&facet=dst"

info = requests.get(url).json()['records'][:]
info

[{'datasetid': 'us-zip-code-latitude-and-longitude',
  'recordid': '0b26467a7ec2b1a30432f960a680a2948091124c',
  'fields': {'city': 'Nutley',
   'zip': '07110',
   'dst': 1,
   'geopoint': [40.8196, -74.15877],
   'longitude': -74.15877,
   'state': 'NJ',
   'latitude': 40.8196,
   'timezone': -5},
  'geometry': {'type': 'Point', 'coordinates': [-74.15877, 40.8196]},
  'record_timestamp': '2018-02-09T16:33:38.603000+00:00'},
 {'datasetid': 'us-zip-code-latitude-and-longitude',
  'recordid': '5913811ba37cf921d5f6f87fefaf472a40167a7f',
  'fields': {'city': 'Bloomfield',
   'zip': '07003',
   'dst': 1,
   'geopoint': [40.803, -74.18895],
   'longitude': -74.18895,
   'state': 'NJ',
   'latitude': 40.803,
   'timezone': -5},
  'geometry': {'type': 'Point', 'coordinates': [-74.18895, 40.803]},
  'record_timestamp': '2018-02-09T16:33:38.603000+00:00'},
 {'datasetid': 'us-zip-code-latitude-and-longitude',
  'recordid': '057eebe89e1cf8031c804bba135441c57b08a634',
  'fields': {'city': 'Montclai

### Parsing the JSON file into a pandas library
#####  Thank you opendatasoft.com for providing this data :)

In [2]:
zips =[]
cities=[]
latitudes=[]
longitudes=[]

for i in range(0,len(info)):
    temp = info[i]['fields']
    cities.append(temp['city'])
    zips.append(temp['zip'])
    longitudes.append(temp['longitude'])
    latitudes.append(temp['latitude'])
    
df = pd.DataFrame({
    "Zip":zips,
    "City":cities,
    "Latitude":latitudes,
    "Longitude":longitudes
                  })

df.head()

,Zip,City,Latitude,Longitude
0,07110,Nutley,40.819600,-74.15877
1,07003,Bloomfield,40.803000,-74.18895
2,07043,Montclair,40.844199,-74.20218
3,08608,Trenton,40.219158,-74.76481
4,08528,Kingston,40.377173,-74.61538


### Parsing the table from wikipedia about the cities in hudson county
##### along with other data we don't need.

In [3]:
hudsonURL = "https://en.wikipedia.org/wiki/Hudson_County,_New_Jersey"
hudsonTables = pd.read_html(hudsonURL)
hudsonCities = hudsonTables[1]
hudsonCities.rename({"Municipality":"City"},axis=1,inplace=True)
hudsonCities.head()

,City,Mapkey,Mun.type,Pop.,Housingunits,Totalarea,Waterarea,Landarea,Pop.density,Housingdensity,School district
0,Bayonne,1.0,city,63024,27799,11.08,5.28,5.80,10858.3,4789.4,Bayonne
1,East Newark,10.0,borough,2406,794,0.12,0.02,0.10,23532.1,7765.8,Harrison (9-12) (S/R)East Newark (K-8)
2,Guttenberg,6.0,town,11176,4839,0.24,0.05,0.20,57116.0,24730.2,North Bergen (9-12) (S/R)Guttenberg (PK-8)
3,Harrison,9.0,town,13620,5228,1.32,0.12,1.20,11319.3,4344.9,Harrison
4,Hoboken,3.0,city,50005,26855,2.01,0.74,1.28,39212.0,21058.7,Hoboken


### Parsing another table from wikipedia about the cities in bergen county.

In [20]:
bergenURL = "https://en.wikipedia.org/wiki/Bergen_County,_New_Jersey#Municipalities"
bergenTables = pd.read_html(bergenURL)

# bergenTables has alot of tables... The table I need has the fist column as 'Municipality(with map key)'
for table in bergenTables:
    if(table.columns[0] == "Municipality(with map key)"):
        bergenCities = table

bergenCities.rename({'Municipality(with map key)':"City"}, axis=1,inplace=True)
bergenCities.head()

,City,Municipaltype,Population,HousingUnits,TotalArea,WaterArea,LandArea,Pop.Density,HousingDensity,Communities[205]
0,Allendale,borough,6505,2388,3.12,0.02,3.10,2100.7,771.2,NaN
1,Alpine,borough,1849,670,9.23,2.82,6.41,288.4,104.5,NaN
2,Bergenfield,borough,26764,9200,2.89,0.01,2.88,9306.5,3199.1,NaN
3,Bogota,borough,8187,2888,0.81,0.05,0.76,10702.5,3775.4,NaN
4,Carlstadt,borough,6127,2495,4.24,0.24,4.00,1532.1,623.9,NaN


### Combining the cities from hudson and bergen county

In [5]:
#cities = bergenCities[['City']]
cities = hudsonCities[['City']]
cities.head()

,City
0,Bayonne
1,East Newark
2,Guttenberg
3,Harrison
4,Hoboken


### How many cities are in bergen and hudson county?

In [6]:
print(hudsonCities.shape)
print(cities.shape)

(13, 11)
(13, 1)


### This will keep cities in hudson county in the zipcode-lat-lng dataframe.

In [7]:
df=df.reset_index()
cities=cities.reset_index()
df=df[ df['City'].isin(cities['City']) ]
df['City'].value_counts()


Jersey City      13
Kearny            2
Union City        2
Secaucus          2
West New York     1
Bayonne           1
Harrison          1
Hoboken           1
North Bergen      1
Name: City, dtype: int64

In [8]:
cities.head()

,index,City
0,0,Bayonne
1,1,East Newark
2,2,Guttenberg
3,3,Harrison
4,4,Hoboken


### Drppping index

In [9]:
cities.drop('index',axis=1,inplace=True)
cities.head()

,City
0,Bayonne
1,East Newark
2,Guttenberg
3,Harrison
4,Hoboken


In [10]:
df.shape

(24, 5)

### Creating a column that includes the zipcode and city

In [11]:
df['Zip'].astype("str")
df['Zipcode City'] = df['Zip'] + ' ' + df['City']
df.head()

,index,Zip,City,Latitude,Longitude,Zipcode City
8,8,07094,Secaucus,40.788500,-74.060000,07094 Secaucus
115,115,07309,Jersey City,40.732760,-74.075485,07309 Jersey City
122,122,07032,Kearny,40.763051,-74.137180,07032 Kearny
124,124,07030,Hoboken,40.744851,-74.032940,07030 Hoboken
208,208,07002,Bayonne,40.666552,-74.117680,07002 Bayonne


### Credentials Cells

In [13]:
CLIENT_ID = "redacted"
CLIENT_SECRET = 'redacted'
VERSION = "20180605"
LIMIT = 100

### Defining method to call the api and parse the response

In [17]:
def getNearbyVenues(names,latitudes,longitudes, radius=700):

    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        
        # Forming the URL based on the current lat,lng
        url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT
        )
        
        # Getting the JSON response
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # Appending useful data to the list venues_list
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    # Converting the nearby
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list
    ])
        
    nearby_venues.columns = ['City',
                          'City Latitude',
                         'City Latitude',
                         'Venue',
                         'Venue Latitude',
                          'Venue Longitude',
                         'Venue Category']
    return(nearby_venues)

## Getting Nearbyvenues as a pandas df using FourSquare API

In [18]:
closeVenues = getNearbyVenues(names=df['Zipcode City'],latitudes=df['Latitude'],longitudes=df['Longitude'])
closeVenues.head()

,City,City Latitude,City Latitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,07094 Secaucus,40.7885,-74.06,Noches De Colombia,40.790820,-74.058280,Colombian Restaurant
1,07094 Secaucus,40.7885,-74.06,BlackJack Mulligans Secaucus,40.790751,-74.058678,Bar
2,07094 Secaucus,40.7885,-74.06,Green Farmers Market,40.791458,-74.056947,Farmers Market
3,07094 Secaucus,40.7885,-74.06,Plaza Center,40.790789,-74.057342,Shopping Mall
4,07094 Secaucus,40.7885,-74.06,Buchmuller Park,40.790701,-74.056131,Park


## Shape & nUnique

In [20]:
print("Shape",closeVenues.shape)
print('Nunique',closeVenues.nunique())

Shape (1030, 7)
Nunique City                24
City Latitude       18
City Latitude       18
Venue              697
Venue Latitude     789
Venue Longitude    788
Venue Category     179
dtype: int64


## Making dummies categories from the venue category.

In [21]:
njDummies = pd.get_dummies(closeVenues[['Venue Category']], prefix="",prefix_sep="")

njDummies['City'] = closeVenues['City']
columns = [njDummies.columns[-1]] + list(njDummies.columns[:-1])
njDummies = njDummies[columns]
njDummies.head()

,City,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,...,Tunnel,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,07094 Secaucus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,07094 Secaucus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,07094 Secaucus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,07094 Secaucus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,07094 Secaucus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
njDummies.shape

(1030, 180)

## Grouping averages by the city

In [23]:
njGrouped = njDummies.groupby('City').mean().reset_index()
njGrouped.head()

,City,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,...,Tunnel,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,07002 Bayonne,0.02,0.0,0.0,0.02,0.0,0.02,0.02,0.000000,0.060000,...,0.0,0.000000,0.020000,0.02,0.0,0.0,0.0,0.02,0.00,0.00
1,07029 Harrison,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.044444,0.000000,...,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.00,0.00,0.00
2,07030 Hoboken,0.04,0.0,0.0,0.00,0.0,0.00,0.02,0.050000,0.000000,...,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.00,0.01,0.01
3,07032 Kearny,0.00,0.0,0.0,0.00,0.0,0.20,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.00,0.00,0.00
4,07047 North Bergen,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.027778,0.083333,...,0.0,0.027778,0.027778,0.00,0.0,0.0,0.0,0.00,0.00,0.00


## Top 5 venues in each city

In [24]:
topN = 5

for h in njGrouped['City']:
    print("====",h,"====")
    temp = njGrouped[njGrouped['City'] == h].T.reset_index()
    temp.columns=['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(topN))
    print('\n')

==== 07002 Bayonne ====
                  venue  freq
0                  Bank  0.06
1              Pharmacy  0.06
2    Italian Restaurant  0.04
3        Ice Cream Shop  0.04
4  Gym / Fitness Center  0.04


==== 07029 Harrison ====
                  venue  freq
0    Chinese Restaurant  0.07
1                   Bar  0.07
2        Ice Cream Shop  0.04
3  Fast Food Restaurant  0.04
4            Donut Shop  0.04


==== 07030 Hoboken ====
                 venue  freq
0   Italian Restaurant  0.07
1          Pizza Place  0.05
2               Bakery  0.05
3  American Restaurant  0.04
4            Pet Store  0.04


==== 07032 Kearny ====
                venue  freq
0           BBQ Joint   0.2
1       Deli / Bodega   0.2
2        Dance Studio   0.2
3  Chinese Restaurant   0.2
4         Pizza Place   0.2


==== 07047 North Bergen ====
                       venue  freq
0  Latin American Restaurant  0.14
1                       Bank  0.08
2           Cuban Restaurant  0.08
3                 Donut S

In [25]:
def return_most_common_venues(row,num_top_venues):
    rowCategories = row.iloc[1:]
    rowCategoriesSorted = rowCategories.sort_values(ascending=False)
    return rowCategoriesSorted.index.values[0:num_top_venues]

### Sorting the most common venues in the dataframe

In [26]:
num_top_venues = 10
indicators = ['st','nd','rd']

columns = ['City']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
cityVenueSorted = pd.DataFrame(columns=columns)
cityVenueSorted['City'] = njGrouped['City']

for ind in np.arange(njGrouped.shape[0]):
    cityVenueSorted.iloc[ind,1:] = return_most_common_venues(njGrouped.iloc[ind,:],num_top_venues)
    
cityVenueSorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,07002 Bayonne,Bank,Pharmacy,Italian Restaurant,Deli / Bodega,Gym / Fitness Center,Pizza Place,Sandwich Place,Cosmetics Shop,Bar,Ice Cream Shop
1,07029 Harrison,Chinese Restaurant,Bar,Ice Cream Shop,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Bakery,Donut Shop,Pizza Place,Pharmacy
2,07030 Hoboken,Italian Restaurant,Bakery,Pizza Place,American Restaurant,Pet Store,Coffee Shop,Gym / Fitness Center,Pub,Gym,Deli / Bodega
3,07032 Kearny,Deli / Bodega,Dance Studio,BBQ Joint,Chinese Restaurant,Pizza Place,Yoga Studio,Fish & Chips Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
4,07047 North Bergen,Latin American Restaurant,Cuban Restaurant,Bank,Ice Cream Shop,Donut Shop,Fast Food Restaurant,Pizza Place,South American Restaurant,Chinese Restaurant,Mobile Phone Shop


### Using KMeans to cluster the areas

In [27]:
from sklearn.cluster import KMeans
k = 5
njGroupedClustering = njGrouped.drop('City',axis=1)
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(njGroupedClustering)


kmeans.labels_[0:10]

array([2, 2, 2, 3, 0, 0, 0, 0, 2, 1], dtype=int32)

## Inserting the Cluster Labels into the DF
#### Creating a df that is the merged dataframe of all the data

In [28]:

cityVenueSorted.insert(0,'Cluster Labels', kmeans.labels_)

njMerged = df
njMerged.head()
njMerged =  njMerged.join(cityVenueSorted.set_index('City'),on='Zipcode City')

njMerged.head()

,Zip,City,Latitude,Longitude,Zipcode City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,07094,Secaucus,40.788500,-74.060000,07094 Secaucus,2,Bank,Liquor Store,Indian Restaurant,Clothing Store,Bar,Italian Restaurant,Kids Store,Deli / Bodega,Cosmetics Shop,Sandwich Place
115,07309,Jersey City,40.732760,-74.075485,07309 Jersey City,1,Italian Restaurant,Mexican Restaurant,Deli / Bodega,Clothing Store,Middle Eastern Restaurant,Donut Shop,Seafood Restaurant,Liquor Store,Gun Range,Cupcake Shop
122,07032,Kearny,40.763051,-74.137180,07032 Kearny,3,Deli / Bodega,Dance Studio,BBQ Joint,Chinese Restaurant,Pizza Place,Yoga Studio,Fish & Chips Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
124,07030,Hoboken,40.744851,-74.032940,07030 Hoboken,2,Italian Restaurant,Bakery,Pizza Place,American Restaurant,Pet Store,Coffee Shop,Gym / Fitness Center,Pub,Gym,Deli / Bodega
208,07002,Bayonne,40.666552,-74.117680,07002 Bayonne,2,Bank,Pharmacy,Italian Restaurant,Deli / Bodega,Gym / Fitness Center,Pizza Place,Sandwich Place,Cosmetics Shop,Bar,Ice Cream Shop


In [29]:
!pip install folium
import folium

     |████████████████████████████████| 94 kB 4.6 MB/s  eta 0:00:01


In [30]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[40.8721589,-74.1240192],zoom_start=11)

x = np.arange(k)
ys = [i * x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat,lon,poi,cluster in zip(njMerged['Latitude'],njMerged['Longitude'],njMerged['Zipcode City'],njMerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius = 5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
        
map_clusters

### Cluster 0

In [32]:
njMerged.loc[njMerged['Cluster Labels']==0, njMerged.columns[[4] +list(range(5,njMerged.shape[1]))]]

,Zipcode City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
301,07093 West New York,0,Latin American Restaurant,Cuban Restaurant,Pizza Place,Grocery Store,Bank,Convenience Store,Café,Chinese Restaurant,Tapas Restaurant,Mobile Phone Shop
405,07087 Union City,0,Latin American Restaurant,Pizza Place,Deli / Bodega,Mexican Restaurant,Liquor Store,Plaza,Donut Shop,South American Restaurant,Bus Line,Mobile Phone Shop
491,07047 North Bergen,0,Latin American Restaurant,Cuban Restaurant,Bank,Ice Cream Shop,Donut Shop,Fast Food Restaurant,Pizza Place,South American Restaurant,Chinese Restaurant,Mobile Phone Shop
734,07086 Union City,0,Bus Line,Pizza Place,Donut Shop,Scenic Lookout,Park,Mexican Restaurant,Bank,Latin American Restaurant,Cuban Restaurant,Bakery


## Cluster 1

In [33]:
njMerged.loc[njMerged['Cluster Labels']==1, njMerged.columns[[4] +list(range(5,njMerged.shape[1]))]]

,Zipcode City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
115,07309 Jersey City,1,Italian Restaurant,Mexican Restaurant,Deli / Bodega,Clothing Store,Middle Eastern Restaurant,Donut Shop,Seafood Restaurant,Liquor Store,Gun Range,Cupcake Shop
258,07303 Jersey City,1,Italian Restaurant,Mexican Restaurant,Deli / Bodega,Clothing Store,Middle Eastern Restaurant,Donut Shop,Seafood Restaurant,Liquor Store,Gun Range,Cupcake Shop
414,07311 Jersey City,1,Italian Restaurant,Middle Eastern Restaurant,Korean Restaurant,Pool Hall,Filipino Restaurant,Fast Food Restaurant,Seafood Restaurant,Liquor Store,Gun Range,Pharmacy
429,07399 Jersey City,1,Italian Restaurant,Middle Eastern Restaurant,Korean Restaurant,Pool Hall,Filipino Restaurant,Fast Food Restaurant,Seafood Restaurant,Liquor Store,Gun Range,Pharmacy
432,07096 Secaucus,1,Italian Restaurant,Mexican Restaurant,Deli / Bodega,Clothing Store,Middle Eastern Restaurant,Donut Shop,Seafood Restaurant,Liquor Store,Gun Range,Cupcake Shop
570,07097 Jersey City,1,Italian Restaurant,Mexican Restaurant,Deli / Bodega,Clothing Store,Middle Eastern Restaurant,Donut Shop,Seafood Restaurant,Liquor Store,Gun Range,Cupcake Shop
631,07308 Jersey City,1,Italian Restaurant,Mexican Restaurant,Deli / Bodega,Clothing Store,Middle Eastern Restaurant,Donut Shop,Seafood Restaurant,Liquor Store,Gun Range,Cupcake Shop
679,07099 Kearny,1,Italian Restaurant,Mexican Restaurant,Deli / Bodega,Clothing Store,Middle Eastern Restaurant,Donut Shop,Seafood Restaurant,Liquor Store,Gun Range,Cupcake Shop


## Cluster 2

In [34]:
njMerged.loc[njMerged['Cluster Labels']==2, njMerged.columns[[4] +list(range(5,njMerged.shape[1]))]]

,Zipcode City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,07094 Secaucus,2,Bank,Liquor Store,Indian Restaurant,Clothing Store,Bar,Italian Restaurant,Kids Store,Deli / Bodega,Cosmetics Shop,Sandwich Place
124,07030 Hoboken,2,Italian Restaurant,Bakery,Pizza Place,American Restaurant,Pet Store,Coffee Shop,Gym / Fitness Center,Pub,Gym,Deli / Bodega
208,07002 Bayonne,2,Bank,Pharmacy,Italian Restaurant,Deli / Bodega,Gym / Fitness Center,Pizza Place,Sandwich Place,Cosmetics Shop,Bar,Ice Cream Shop
295,07029 Harrison,2,Chinese Restaurant,Bar,Ice Cream Shop,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Bakery,Donut Shop,Pizza Place,Pharmacy
297,07307 Jersey City,2,Pizza Place,Pharmacy,Italian Restaurant,Café,Cuban Restaurant,Latin American Restaurant,Bakery,Bar,Farmers Market,Donut Shop
406,07310 Jersey City,2,Gym,Art Gallery,Cosmetics Shop,Wine Shop,Fast Food Restaurant,Donut Shop,Café,Thrift / Vintage Store,Ice Cream Shop,Furniture / Home Store
441,07302 Jersey City,2,Bar,Bakery,Pizza Place,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Gym,American Restaurant,Café,Grocery Store
666,07305 Jersey City,2,Deli / Bodega,Pharmacy,Bus Stop,Gym,Donut Shop,Salon / Barbershop,Fried Chicken Joint,Mexican Restaurant,Bridal Shop,Breakfast Spot
726,07304 Jersey City,2,Deli / Bodega,Pizza Place,Pharmacy,Convenience Store,Fast Food Restaurant,Discount Store,Supermarket,Dessert Shop,Park,Cosmetics Shop
744,07301 Jersey City,2,Cosmetics Shop,Café,Waterfront,American Restaurant,Coffee Shop,Mexican Restaurant,Fried Chicken Joint,Fast Food Restaurant,Scenic Lookout,Restaurant


## Cluster 3

In [35]:
njMerged.loc[njMerged['Cluster Labels']==3, njMerged.columns[[4] +list(range(5,njMerged.shape[1]))]]

,Zipcode City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
122,07032 Kearny,3,Deli / Bodega,Dance Studio,BBQ Joint,Chinese Restaurant,Pizza Place,Yoga Studio,Fish & Chips Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint


In [ ]:
njMerged.loc[njMerged['Cluster Labels']==4, njMerged.columns[[4] +list(range(5,njMerged.shape[1]))]]